In [1]:
from functools import partial

!date
!whoami
!uname -a
!pwd

Sat Nov  5 11:30:44 PDT 2022
ndbs
Darwin ndbs-Q9Q94K6GHF 21.6.0 Darwin Kernel Version 21.6.0: Mon Aug 22 20:19:52 PDT 2022; root:xnu-8020.140.49~2/RELEASE_ARM64_T6000 arm64
/Users/ndbs/notebooks


# Goal

## Write a second-order function similar to `functools.partial` that returns a partially applied function, but using synatx more similar to the notation mathematicians use

* https://en.wikipedia.org/wiki/Partial_application

* https://docs.python.org/3/library/functools.html#functools.partial

Specifically, the idea is to use the `Ellipsis` object as a placeholder for arguments being treated as variables, similar to how mathematicians use `\cdot` or `-`.
For example, I want `f(..., 3, ..., loc=4, scale=...)` to return a function `g` of three arguments such that `g(x,y,z)` or `g(x,y,scale=z)` equals `f(x, 3, y, loc=4, scale=z)`, so that `g` is a partially applied version of `f`.

## Approximate implementation of `functools.partial`, for reference

Code from https://docs.python.org/3/library/functools.html#functools.partial

In [294]:
def fake_partial(func, /, *args, **keywords):
    def newfunc(*fargs, **fkeywords):
        # Note: Creating a dictionary union overwrites keywords (passed on creation)
        # with fkeywords (passed when calling the partial function) if the same
        # keyword is passed in both places; passing (..., **keywords, **fkeywords) directly
        # to func would instead raise a TypeError in this case.
        newkeywords = {**keywords, **fkeywords}
        return func(*args, *fargs, **newkeywords)
    newfunc.func = func
    newfunc.args = args
    newfunc.keywords = keywords
    return newfunc

# Define some functions, and test `functools.partial` on them

In [296]:
def f1(a, b, c, d=1, e=2):
    print(f'{a=},{b=},{c=},{d=},{e=}')

def g1(*args, **kwargs):
    print(args, kwargs)

## Apparently it's not possible to specify the value of one positional argument that comes before another positional argument that you want to leave unspecified, unless you pass the second argument as a keyword argument...

In [297]:
f1a=partial(f1, 1, b=2, e=4)
f1a

functools.partial(<function f1 at 0x10828fa60>, 1, b=2, e=4)

In [303]:
f1a() # This error is to be expected

TypeError: f1() missing 1 required positional argument: 'c'

In [304]:
# This is supposed to be the value of c, not the value of b...
f1a(4)

TypeError: f1() got multiple values for argument 'b'

In [305]:
# I guess you have to pass c as a keyword argument
f1a(c=4)

a=1,b=2,c=4,d=1,e=4


In [298]:
# This is a confusing error...
f1a(3,4)

TypeError: f1() got multiple values for argument 'b'

## Try again, but leave both `a` and `c` unspecified

In [306]:
f1b = partial(f1, b=2, e=4)

In [309]:
f1b(1) # This error is expected

TypeError: f1() missing 1 required positional argument: 'c'

In [310]:
f1b(1,2) # This error is annoying, but perhaps to be expected

TypeError: f1() got multiple values for argument 'b'

In [311]:
f1b(1,c=2) # Again, we have to specify c as a keyword argument

a=1,b=2,c=2,d=1,e=4


In [320]:
f1b(8,b=4,c=12)

a=8,b=4,c=12,d=1,e=4


## Try another example

In [13]:
f1c=partial(f1, 1, 2, e=4)
f1c

functools.partial(<function f1 at 0x1074c2a60>, 1, 2, e=4)

In [16]:
f1c(4,3)

a=1,b=2,c=4,d=3,e=4


In [15]:
f1c(8,d=5)

a=1,b=2,c=8,d=5,e=4


In [19]:
f1c.func

<function __main__.f1(a, b, c, d=1, e=2)>

In [18]:
f1c.args

(1, 2)

In [20]:
f1c.keywords

{'e': 4}

In [23]:
f1.__name__

'f1'

In [24]:
f1b.__name__

AttributeError: 'functools.partial' object has no attribute '__name__'

# What about `g`?

In [313]:
g1a = partial(g1) # This returns a function equal to g
g1a

functools.partial(<function g1 at 0x107eff160>)

In [314]:
g1a()

() {}


In [321]:
g1a(9)

(9,) {}


In [322]:
g1a(9, x=398)

(9,) {'x': 398}


In [315]:
g1b = partial(g1, 112, 3,4, x=5, y=7)
g1b

functools.partial(<function g1 at 0x107eff160>, 112, 3, 4, x=5, y=7)

In [316]:
g1b(1,2,3)

(112, 3, 4, 1, 2, 3) {'x': 5, 'y': 7}


In [317]:
g1b(1,2, z=4, w=238)

(112, 3, 4, 1, 2) {'x': 5, 'y': 7, 'z': 4, 'w': 238}


In [318]:
g1b(999,x=5)

(112, 3, 4, 999) {'x': 5, 'y': 7}


In [319]:
g1b(999,x=27)

(112, 3, 4, 999) {'x': 27, 'y': 7}


# Try writing my own version of partially applied functions, using `...` as a placeholder to identify positions of variables

## For simplicity, first write a version that only accepts positional arguments

First I need a way to identify the positions of arguments that were passed an `Ellipsis`

In [295]:
# .index() only finds the first position
t = (2,3,4,2,5,6,4,8,9)
t.index(4)

2

In [51]:
# Haha, this works! Thanks, walrus!
args = (3,...,5,4,...)
count=0
omitted_indices = {i: (count:=count+1)-1 for i, arg in enumerate(args) if arg is SENTINEL}
omitted_indices

{1: 0, 4: 1}

In [80]:
SENTINEL = Ellipsis

def partially_applicable_no_kwargs(func):
#     def partially_applied_func(*new_args):
#         if SENTINEL in new_args:
#             raise ValueError(f"{SENTINEL} is not a valid argument.")
#         return
    
    def new_func(*args):
#     First attempt:
#         omitted_indices = [i for i, arg in enumerate(args) if arg is SENTINEL]
#         omitted_indices = {i:j for j,i in enumerate(omitted_indices)}
#     Second attempt:
#         omitted_indices = {}
#         count = 0
#         for i, arg in enumerate(args):
#             omitted_indices[i] = count
#             count += 1
#     Third attempt:
        # Map omitted indices to a new index in range(len(omitted_indices))
        count=0
        omitted_indices = {i: (count:=count+1)-1 for i, arg in enumerate(args) if arg is SENTINEL}
        print(f"{omitted_indices=}")
        if len(omitted_indices) == 0:
            return func(*args) # Nullary function because arity equals number of omitted indices
        
#         fixed_args = [(i, args[i]) for i not in omitted_indices]
        def partially_applied_func(*fewer_args): # Define a function of arity len(omitted_indices)
            print(f"{omitted_indices=}, {args=}, {fewer_args=}")
            new_args = []
            for i, arg in enumerate(args):
                new_args.append(fewer_args[omitted_indices[i]] if i in omitted_indices else arg)
            
#             new_args = [fewer_args[omitted_indices[i]] if i in omitted_indices else arg
#                         for i, arg in enumerate(args)]
            return func(*new_args)
        
        return partially_applied_func
    return new_func

In [138]:
def f2(a,b,c):
    print(f'{a=},{b=},{c=}')

pf2 = partially_applicable_no_kwargs(f2)
pf2

<function __main__.partially_applicable_no_kwargs.<locals>.new_func(*args)>

In [139]:
pf2(3,5,7)()

omitted_indices={}
omitted_indices={}, args=(3, 5, 7), fewer_args=()
new_args=[3, 5, 7]
a=3,b=5,c=7


In [117]:
pf2(...,2,3)

omitted_indices={0: 0}


<function __main__.partially_applicable_no_kwargs.<locals>.new_func.<locals>.partially_applied_func(*fewer_args)>

In [118]:
pf2(...,2,3)(4)

omitted_indices={0: 0}
omitted_indices={0: 0}, args=(Ellipsis, 2, 3), fewer_args=(4,)
a=4,b=2,c=3


In [119]:
pf2(5,...,3)

omitted_indices={1: 0}


<function __main__.partially_applicable_no_kwargs.<locals>.new_func.<locals>.partially_applied_func(*fewer_args)>

In [120]:
pf2(5,...,3)(8)

omitted_indices={1: 0}
omitted_indices={1: 0}, args=(5, Ellipsis, 3), fewer_args=(8,)
a=5,b=8,c=3


In [121]:
pf2(...,5,...)(2,3)

omitted_indices={0: 0, 2: 1}
omitted_indices={0: 0, 2: 1}, args=(Ellipsis, 5, Ellipsis), fewer_args=(2, 3)
a=2,b=5,c=3


In [122]:
# "Chaining" doesn't work because the returned function is not partially applicable
pf2(...,5,...)(2,...)

omitted_indices={0: 0, 2: 1}
omitted_indices={0: 0, 2: 1}, args=(Ellipsis, 5, Ellipsis), fewer_args=(2, Ellipsis)
a=2,b=5,c=Ellipsis


# "Chaining" didn't work like I had hoped... let's try again, using recursion

That is, with the above code, I can't pass an `Ellipsis` to a partially applicable function to get another partially applicable function.

## First, just copy the above version, but clean it up

In [240]:
SENTINEL = Ellipsis

def partially_applicable_no_kwargs(func):
    def new_func(*args):
#     Third attempt:
        # Map omitted indices to a new index in range(len(omitted_indices))
        count=0
        omitted_indices = {i: (count:=count+1)-1 for i, arg in enumerate(args) if arg is SENTINEL}
        print(f"{omitted_indices=}")
#         if len(omitted_indices) == 0:
#             return func(*args) # Nullary function because arity equals number of omitted indices
        
        def partially_applied_func(*fewer_args): # Define a function of arity len(omitted_indices)
            print(f"{omitted_indices=}, {args=}, {fewer_args=}")
            new_args = [fewer_args[omitted_indices[i]] if i in omitted_indices else arg
                        for i, arg in enumerate(args)]
            print(f"{new_args=}")
            return func(*new_args)
        
        return partially_applied_func
    return new_func

## Do we actually need *nested* inner functions, or could we do it with just one??

## It seems like we need two because we need two places to pass arguments...

**Yes**, the first inner function is a function wapper that turns the pased function `func` into a partially applicable function. The second inner function (the one inside the function wrapper) is what you get when you _call_ the partially applicable function, i.e. the partially _applied_ function; this is what the partially applicable function returns -- a partially applied version of `func` (which could be a "totally applied" version of `func` if you pass all the arguments).

**Note:** We could instead implement "partially applicable function" inner function as a _class_, which would probably be more flexible, especially since it would be subclassable, allowing more features to be added in the future. Such as being able to pass positional arguments to a partially applicable function as keyword arguments, which I think is more trouble than it's worth on a first pass because I think it [requires](https://stackoverflow.com/questions/42352703/get-names-of-positional-arguments-from-functions-signature) the `inspect` module. But I _do_ want to be able to pass keyword arguments as positional arguments, which should be easier, but I haven't quite figured out how to do it as of 11/6/2022.

In [235]:
def partially_applicable_no_kwargs1(func, nullary_as_const=True):
    def partially_applicable_func_no_kwargs(*args):
        # This list maps the index of each new argument to its original position in args
        omitted_indices = [j for j, arg in enumerate(args) if arg is SENTINEL]
#   This works, but then we don't have the option to return a function rather than a constant:
#         if len(omitted_indices) == 0:
#             return func(*args) # Nullary function because arity equals number of omitted indices
        print(f"{omitted_indices=}")
        def partially_applied_func(*fewer_args):
            print(f"{omitted_indices=}, {args=}, {fewer_args=}")
            if len(fewer_args) != len(omitted_indices):
                raise TypeError(
                    f"The partially applied function takes {len(omitted_indices)} "
                    f"positional arguments, but {len(fewer_args)} were given"
                )
            new_args = list(args)
            for i,j in enumerate(omitted_indices):
                new_args[j] = fewer_args[i]
            print(f"{new_args=}")
            return func(*new_args)
        
        if len(omitted_indices) == 0:
            # Base case: Don't recurse when result is nullary
            return partially_applied_func() if nullary_as_const else partially_applied_func
        else:
            # If not nullary, use recursion to get another partially applicable function
            return partially_applicable_no_kwargs1(partially_applied_func)
        
    return partially_applicable_func_no_kwargs

In [236]:
# By default, evaluate f when all arguments are supplied
pf2a = partially_applicable_no_kwargs1(f2)
pf2a

<function __main__.partially_applicable_no_kwargs1.<locals>.partially_applicable_func_no_kwargs(*args)>

In [239]:
pf2a(3,4,5)

omitted_indices=[]
omitted_indices=[], args=(3, 4, 5), fewer_args=()
new_args=[3, 4, 5]
a=3,b=4,c=5


In [221]:
# Specify nullary_as_const=False to return a nullay function without evaluating it automatically
pf2b = partially_applicable_no_kwargs1(f2, False)
pf2b

<function __main__.partially_applicable_no_kwargs1.<locals>.partially_applicable_func_no_kwargs(*args)>

In [222]:
pf2b(3,4,5)

omitted_indices=[]


<function __main__.partially_applicable_no_kwargs1.<locals>.partially_applicable_func_no_kwargs.<locals>.partially_applied_func(*fewer_args)>

In [223]:
pf2b(3,4,5)()

omitted_indices=[]
omitted_indices=[], args=(3, 4, 5), fewer_args=()
new_args=[3, 4, 5]
a=3,b=4,c=5


In [224]:
pf2a(3,4,...)(7)

omitted_indices=[2]
omitted_indices=[]
omitted_indices=[], args=(7,), fewer_args=()
new_args=[7]
omitted_indices=[2], args=(3, 4, Ellipsis), fewer_args=(7,)
new_args=[3, 4, 7]
a=3,b=4,c=7


In [229]:
pf2a(...,4,...)(7,9)

omitted_indices=[0, 2]
omitted_indices=[]
omitted_indices=[], args=(7, 9), fewer_args=()
new_args=[7, 9]
omitted_indices=[0, 2], args=(Ellipsis, 4, Ellipsis), fewer_args=(7, 9)
new_args=[7, 4, 9]
a=7,b=4,c=9


In [230]:
pf2a(...,4,...)(7,...)

omitted_indices=[0, 2]
omitted_indices=[1]


<function __main__.partially_applicable_no_kwargs1.<locals>.partially_applicable_func_no_kwargs(*args)>

In [234]:
# Haha! "Chaining" now works like I wanted, using recursion
pf2a(...,4,...)(7,...)(12)

omitted_indices=[0, 2]
omitted_indices=[1]
omitted_indices=[]
omitted_indices=[], args=(12,), fewer_args=()
new_args=[12]
omitted_indices=[1], args=(7, Ellipsis), fewer_args=(12,)
new_args=[7, 12]
omitted_indices=[0, 2], args=(Ellipsis, 4, Ellipsis), fewer_args=(7, 12)
new_args=[7, 4, 12]
a=7,b=4,c=12


# Now let's try adding keyword arguments!

In [277]:
def partially_applicable(func, nullary_as_const=True):
    def partially_applicable_func(*args, **kwargs):
        # This list maps the index of each new argument to its original position in args
        omitted_indices = [j for j, arg in enumerate(args) if arg is SENTINEL]
        omitted_kwds = [kwd for kwd, val in kwargs.items() if val is SENTINEL]
        print(f"{omitted_indices=}, {omitted_kwds=}")
        
        def partially_applied_func(*fewer_args, **fewer_kwargs):
            print(f"{omitted_indices=}, {args=}, {fewer_args=}")
            print(f"{omitted_kwds=}, {kwargs=}, {fewer_kwargs=}")
            if (n_given:=len(fewer_args)+len(fewer_kwargs)) != (n_required:=len(omitted_indices)+len(omitted_kwds)):
                raise TypeError(
                    f"The partially applied function takes {n_required} "
                    f"arguments, but {n_given} were given"
                )
            new_args, new_kwargs = list(args), dict(kwargs)
            
            for i,j in enumerate(omitted_indices):
                new_args[j] = fewer_args[i]
            for kwd in omitted_kwds:
                new_kwargs[kwd] = fewer_kwargs[kwd]
            print(f"{new_args=}, {new_kwargs=}")
            return func(*new_args, **new_kwargs)
        
        if len(omitted_indices) == 0 and len(omitted_kwds) == 0:
            # Base case: Don't recurse when result is nullary
            return partially_applied_func() if nullary_as_const else partially_applied_func
        else:
            # If not nullary, use recursion to get another partially applicable function
            return partially_applicable(partially_applied_func)
        
    return partially_applicable_func

In [278]:
d = dict(a=4,b=5)
d

{'a': 4, 'b': 5}

In [279]:
def f3(a,b,c, d=10, e=47):
    print(f'{a=},{b=},{c=},{d=},{e=}')
f3(1,4,3)

a=1,b=4,c=3,d=10,e=47


In [280]:
pf3a = partially_applicable(f3)
pf3a

<function __main__.partially_applicable.<locals>.partially_applicable_func(*args, **kwargs)>

In [281]:
pf3a(1,2,3)

omitted_indices=[], omitted_kwds=[]
omitted_indices=[], args=(1, 2, 3), fewer_args=()
omitted_kwds=[], kwargs={}, fewer_kwargs={}
new_args=[1, 2, 3], new_kwargs={}
a=1,b=2,c=3,d=10,e=47


In [282]:
pf3a(1,2,3,4)

omitted_indices=[], omitted_kwds=[]
omitted_indices=[], args=(1, 2, 3, 4), fewer_args=()
omitted_kwds=[], kwargs={}, fewer_kwargs={}
new_args=[1, 2, 3, 4], new_kwargs={}
a=1,b=2,c=3,d=4,e=47


In [283]:
pf3a(1,...,3,4)(5)

omitted_indices=[1], omitted_kwds=[]
omitted_indices=[], omitted_kwds=[]
omitted_indices=[], args=(5,), fewer_args=()
omitted_kwds=[], kwargs={}, fewer_kwargs={}
new_args=[5], new_kwargs={}
omitted_indices=[1], args=(1, Ellipsis, 3, 4), fewer_args=(5,)
omitted_kwds=[], kwargs={}, fewer_kwargs={}
new_args=[1, 5, 3, 4], new_kwargs={}
a=1,b=5,c=3,d=4,e=47


In [284]:
pf3a(1,...,3,...)(5,6)

omitted_indices=[1, 3], omitted_kwds=[]
omitted_indices=[], omitted_kwds=[]
omitted_indices=[], args=(5, 6), fewer_args=()
omitted_kwds=[], kwargs={}, fewer_kwargs={}
new_args=[5, 6], new_kwargs={}
omitted_indices=[1, 3], args=(1, Ellipsis, 3, Ellipsis), fewer_args=(5, 6)
omitted_kwds=[], kwargs={}, fewer_kwargs={}
new_args=[1, 5, 3, 6], new_kwargs={}
a=1,b=5,c=3,d=6,e=47


In [285]:
pf3a(1,...,3,d=...)(5,d=6)

omitted_indices=[1], omitted_kwds=['d']
omitted_indices=[], omitted_kwds=[]
omitted_indices=[], args=(5,), fewer_args=()
omitted_kwds=[], kwargs={'d': 6}, fewer_kwargs={}
new_args=[5], new_kwargs={'d': 6}
omitted_indices=[1], args=(1, Ellipsis, 3), fewer_args=(5,)
omitted_kwds=['d'], kwargs={'d': Ellipsis}, fewer_kwargs={'d': 6}
new_args=[1, 5, 3], new_kwargs={'d': 6}
a=1,b=5,c=3,d=6,e=47


In [286]:
pf3a(1,...,c=4,d=...)(5,d=12)

omitted_indices=[1], omitted_kwds=['d']
omitted_indices=[], omitted_kwds=[]
omitted_indices=[], args=(5,), fewer_args=()
omitted_kwds=[], kwargs={'d': 12}, fewer_kwargs={}
new_args=[5], new_kwargs={'d': 12}
omitted_indices=[1], args=(1, Ellipsis), fewer_args=(5,)
omitted_kwds=['d'], kwargs={'c': 4, 'd': Ellipsis}, fewer_kwargs={'d': 12}
new_args=[1, 5], new_kwargs={'c': 4, 'd': 12}
a=1,b=5,c=4,d=12,e=47


In [287]:
pf3a(1,...,c=4,d=...)(5,12)

omitted_indices=[1], omitted_kwds=['d']
omitted_indices=[], omitted_kwds=[]
omitted_indices=[], args=(5, 12), fewer_args=()
omitted_kwds=[], kwargs={}, fewer_kwargs={}
new_args=[5, 12], new_kwargs={}
omitted_indices=[1], args=(1, Ellipsis), fewer_args=(5, 12)
omitted_kwds=['d'], kwargs={'c': 4, 'd': Ellipsis}, fewer_kwargs={}


KeyError: 'd'

In [292]:
pf3a(1,...,c=4,d=...,e=2)(5,d=12)

omitted_indices=[1], omitted_kwds=['d']
omitted_indices=[], omitted_kwds=[]
omitted_indices=[], args=(5,), fewer_args=()
omitted_kwds=[], kwargs={'d': 12}, fewer_kwargs={}
new_args=[5], new_kwargs={'d': 12}
omitted_indices=[1], args=(1, Ellipsis), fewer_args=(5,)
omitted_kwds=['d'], kwargs={'c': 4, 'd': Ellipsis, 'e': 2}, fewer_kwargs={'d': 12}
new_args=[1, 5], new_kwargs={'c': 4, 'd': 12, 'e': 2}
a=1,b=5,c=4,d=12,e=2


In [293]:
pf3a(1,b=...,c=4,d=...,e=2)(b=5,d=12)

omitted_indices=[], omitted_kwds=['b', 'd']
omitted_indices=[], omitted_kwds=[]
omitted_indices=[], args=(), fewer_args=()
omitted_kwds=[], kwargs={'b': 5, 'd': 12}, fewer_kwargs={}
new_args=[], new_kwargs={'b': 5, 'd': 12}
omitted_indices=[], args=(1,), fewer_args=()
omitted_kwds=['b', 'd'], kwargs={'b': Ellipsis, 'c': 4, 'd': Ellipsis, 'e': 2}, fewer_kwargs={'b': 5, 'd': 12}
new_args=[1], new_kwargs={'b': 5, 'c': 4, 'd': 12, 'e': 2}
a=1,b=5,c=4,d=12,e=2
